## SSM implementation + testing

TEST - lily

#### general notes

- time series model (note for matplotlib animate visualization)
- "model" how a state space evolves over time
- each discrete time step ∆t corresponds to 1 input data (-> step method)


### prof suggestions

- tweak the $A$ matrix (i.e. padding certian positions with 0 to see which structures are more important than others, or pad 0s at 1,1, or n,n positions of the matrix to see something about long-range dependency)
- possibly include stuff on matrix conditioning? (perfectly-conditioned matrix = unitary)

#### other notes

- explore how linear dynamics/physical systems are reflected by the SSM (S4)
- maybe tweaking $A$ will give us a general idea of how RNN is different from SSM
- if time permits, implement RNN from scratch and explain how the hidden state is processed diffferently

Daniel from gc:
- ...it is similar in format from an RNN except the hidden matrix in RNN is non linear and only trained by gradient descent, whereas state space model like reflects linear dynamics / a more physical system and is more structured 

In [16]:
# import packages

#### Model implementation

things I want to cover:
- classification
- sklearn metrics
- dimensionality reduction
- cross-validation
- hyperparameter tuning


In [17]:
# model implementation

#### Model visualization

ideas to start off w/ (in-progress):

- plot loss function over iterations
- visualize parameter updates
- matplotlib/seaborn
    - attention heatmaps (extract attension weights from model + plot)
- Model architecture/internal states
    - Inspectus: used for ML visualization, explore model internals
    - or just matplotlib (e.g. check activation patterns across model layers/components)
- efficiency-focused visualizations
    - LTSF-Linear: visualization of weights after training, get insight to predicted values
    - OptimusPy: (check what this is)
    - PPT (Token Pruning & Pooling Transformers): (check what this is)

NOTE: We could do similar thing as Gu's Figure 2 (trained S4 model on LRA Path-X, where SSM convolution kernels are reshaped into a 128x128 image)

In [18]:
# visualiation